In [1]:
import torch as t
from collections import namedtuple

import utils

In [2]:
test_input = t.tensor(
    [[0, 1, 2, 3, 4], 
    [5, 6, 7, 8, 9], 
    [10, 11, 12, 13, 14], 
    [15, 16, 17, 18, 19]], dtype=t.float
)

test_input.shape, test_input.stride()

(torch.Size([4, 5]), (5, 1))

In [3]:
TestCase = namedtuple("TestCase", ["output", "size", "stride"])

test_cases = [
    TestCase(
        output=t.tensor([0, 1, 2, 3]), 
        size=(4,), 
        stride=(1,)),
    TestCase(
        output=t.tensor([0, 1, 2, 3, 4]), 
        size=(5,), 
        stride=(1,)),
    TestCase(
        output=t.tensor([0, 5, 10, 15]), 
        size=(4,), 
        stride=(5,)),
    TestCase(
        output=t.tensor([[0, 1, 2], [5, 6, 7]]), 
        size=(2, 3), 
        stride=(5, 1)),
    TestCase(
        output=t.tensor([[0, 1, 2], [10, 11, 12]]), 
        size=(2, 3), 
        stride=(10, 1)),
    TestCase(
        output=t.tensor([[0, 0, 0], [11, 11, 11]]), 
        size=(2, 3), 
        stride=(11, 0)),   
    TestCase(
        output=t.tensor([0, 6, 12, 18]), 
        size=(4,), 
        stride=(6,)),
    TestCase(
        output=t.tensor([[[0, 1, 2]], [[9, 10, 11]]]), 
        size=(2, 1, 3), 
        stride=(9, 0, 1)),
    TestCase(
        output=t.tensor([[[[0, 1], [2, 3]], [[4, 5], [6, 7]]], [[[12, 13], [14, 15]], [[16, 17], [18, 19]]]]),
        size=(2, 2, 2, 2),
        stride=(12, 4, 2, 1)),
]
for (i, case) in enumerate(test_cases):
    if (case.size is None) or (case.stride is None):
        print(f"Test {i} failed: attempt missing.")
    else:
        actual = test_input.as_strided(size=case.size, stride=case.stride)
        if (case.output != actual).any():
            print(f"Test {i} failed:")
            print(f"Expected: {case.output}")
            print(f"Actual: {actual}")
        else:
            print(f"Test {i} passed!")

Test 0 passed!
Test 1 passed!
Test 2 passed!
Test 3 passed!
Test 4 passed!
Test 5 passed!
Test 6 passed!
Test 7 passed!
Test 8 passed!


In [4]:
def as_strided_trace(mat: t.Tensor) -> t.Tensor:
    '''
    Returns the same as `torch.trace`, using only `as_strided` and `sum` methods.
    '''
    n = mat.shape[-1]
    return mat.as_strided(size=(n,), stride=(n+1,)).sum()

utils.test_trace(as_strided_trace)

All tests in `test_trace` passed!


In [5]:
def as_strided_mv(mat: t.Tensor, vec: t.Tensor) -> t.Tensor:
    '''
    Returns the same as `torch.matmul`, using only `as_strided` and `sum` methods.
    '''
    return (mat * vec.as_strided(size=mat.shape, stride=(0, vec.stride()[0]))).sum(dim=1)

utils.test_mv(as_strided_mv)
utils.test_mv2(as_strided_mv)

All tests in `test_mv` passed!
All tests in `test_mv2` passed!


In [6]:
def as_strided_mm(matA: t.Tensor, matB: t.Tensor) -> t.Tensor:
    '''
    Returns the same as `torch.matmul`, using only `as_strided` and `sum` methods.
    '''
    rowA, colA = matA.stride()
    rowB, colB = matB.stride()

    strA = matA.as_strided(size=(matB.shape[1], *matA.shape), stride=(0, rowA, colA))
    strB = matB.as_strided(size=(matB.shape[1], *matA.shape), stride=(colB, 0, rowB))

    return (strA * strB).sum(dim=2).T

utils.test_mm(as_strided_mm)
utils.test_mm2(as_strided_mm)

All tests in `test_mm` passed!
All tests in `test_mm2` passed!
